<a href="https://colab.research.google.com/github/mrab72/user-behave-prediction/blob/master/Internet-access-text-gen-lang-model-LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Connect to the google drive:



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Import required packages

In [ ]:
import json
from collections import defaultdict
import operator
from django.utils.dateparse import parse_datetime
import seaborn as sns
import pandas as pd
import numpy as np
from __future__ import absolute_import, division, print_function, unicode_literals

from matplotlib import pyplot as plt
from datetime import datetime, timedelta

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
tf.compat.v1.enable_eager_execution

import numpy as np
import os
import time
import random
from collections import OrderedDict

## Load first user data

In [ ]:
with open ('/content/drive/My Drive/tubp/file_84_241_9_79_apr.json', 'r') as f:
  data = json.load(f)

## Split Data into the 5_minutes chunks
by majority voting choose, label behavior of each time steps.
here each step is 5 minutes.

### parsed date from data:**bold text**


In [ ]:
parsed_data = {}
for date, traffic in data.items():
  parsed_date = datetime.strptime(date.split('.')[0], '%Y-%m-%d-%H:%M:%S')
  parsed_data[parsed_date] = 1

In [ ]:
ordered_data = OrderedDict(sorted(parsed_data.items(), key=lambda t: t[0]))

### create 5 minutes sequence

In [ ]:
internet_access_seq = OrderedDict(([(datetime(2020, 1, 6, 12, 55), 0)]))
date = datetime(2020, 1, 6, 12, 55)

while date < datetime(2020,3,25,9,36):
  date = date + timedelta(0, 60*15)
  internet_access_seq[date] = 0


In [ ]:
for date, value in ordered_data.items():
  new_date = (date - timedelta(0, 60 * (date.minute  % 15))).replace(second=0)
  internet_access_seq[new_date] = 1

## Predict Internet Access by Text generation method

In [ ]:
traffics = list(internet_access_seq.values())
dates = list(internet_access_seq.keys())

In [ ]:
len(traffics)

14469

### prepare requrired methods for rnn model:

In [ ]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

In [ ]:
def _prepare_dataset(data):
  # dictionary = list(set(data))

  # char2idx = {u:i for i, u in enumerate(dictionary)}
  # idx2char = np.array(dictionary)

  # text_as_int = np.array([char2idx[c] for c in data])

  # The maximum length sentence we want for a single input in characters
  seq_length = 100
  # examples_per_epoch = len(data) // (seq_length+1)

  # Create training examples / targets 
  char_dataset = tf.data.Dataset.from_tensor_slices(data)
  sequences = char_dataset.batch(seq_length + 1, drop_remainder=True)

  dataset = sequences.map(split_input_target)
  return dataset
  

### build model

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):

  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(
        vocab_size,
          embedding_dim,
          batch_input_shape=[batch_size, None]
          ),
    tf.keras.layers.LSTM(rnn_units,
        return_sequences=True,
        stateful=True,
        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [ ]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(
      labels, logits, from_logits=True
      )

### Train the model

In [ ]:
# Batch size
BATCH_SIZE = 1

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 1000


# Length of the vocabulary in chars

# The embedding dimension
embedding_dim = 8

# Number of RNN units
rnn_units = 1024
dataset = _prepare_dataset(traffics[:int(len(traffics)/2)])
time_model = build_model(
  vocab_size = 2,
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE
  )

dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

EPOCHS=10

time_model.compile(optimizer='adam', loss=loss)

history = time_model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
71/71 [==============================] - 65s 918ms/step - loss: 0.0197
Epoch 2/10
71/71 [==============================] - 65s 919ms/step - loss: 0.0000e+00
Epoch 3/10
71/71 [==============================] - 65s 921ms/step - loss: 0.0000e+00
Epoch 4/10
71/71 [==============================] - 65s 912ms/step - loss: 0.0000e+00
Epoch 5/10
71/71 [==============================] - 65s 915ms/step - loss: 0.0000e+00
Epoch 6/10
71/71 [==============================] - 65s 916ms/step - loss: 0.0000e+00
Epoch 7/10
71/71 [==============================] - 69s 974ms/step - loss: 0.0000e+00
Epoch 8/10
71/71 [==============================] - 65s 915ms/step - loss: 0.0000e+00
Epoch 9/10
71/71 [==============================] - 65s 916ms/step - loss: 0.0000e+00
Epoch 10/10
71/71 [==============================] - 65s 916ms/step - loss: 0.0000e+00


### Generate Sequence

In [ ]:
def generate_sequence(model, start_string, num_generate):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = num_generate

  # Converting our start string to numbers (vectorizing)
  input_eval = start_string
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(predicted_id)

  return text_generated

### evaluate model

In [ ]:
next_internet_access = generate_sequence(
    time_model, start_string=traffics[:int(len(traffics)/2)], num_generate=100*5
    )

In [ ]:
true_traffic_access  = traffics[int(len(traffics)/2):int(len(traffics)/2) + 100*5]

#### Accuracy:

In [ ]:
np.sum(np.array(true_traffic_access) == np.array(next_internet_access)) / 500

0.676

#### recall and precision:
![recall and precision](https://en.wikipedia.org/wiki/File:Precisionrecall.svg)

In [ ]:
true_traffic_access = np.array(true_traffic_access)
next_internet_access = np.array(next_internet_access)

##### True Positive(1)(tp):

In [ ]:
predicted_p_index = np.where(next_internet_access == 1)[0]
predicted_n_index = np.where(next_internet_access == 0)[0]

true_p_index = np.where(true_traffic_access == 1)[0]
true_n_index = np.where(true_traffic_access == 0)[0]

In [ ]:
tp = 0
for item in predicted_p_index:
  if item in true_p_index:
    tp += 1

##### False Positive(1)(fp):

In [ ]:
tp = 0
for item in predicted_p_index:
  if item in true_p_index:
    tp += 1
fp = 0
for item in predicted_p_index:
  if item not in true_p_index:
    fp += 1
fn = 0
for item in predicted_n_index:
  if item in true_n_index:
    fn += 1
# precision = tp / (tp + fp)
# recall = tp / (tp + fn)


##### False Negative(0)(fn):

In [ ]:
predicted_n_index = np.where(true_traffic_access == 0)[0]
true_n_index = np.where(next_internet_access == 0)[0]

In [ ]:
fn = 0
for item in predicted_n_index:
  if item in true_n_index:
    fn += 1

##### precision:

In [ ]:
tp / (tp + fp)

ZeroDivisionError: ignored

##### recall:

In [ ]:
tp / (tp + fn)

0.0

#### Save Model:

In [ ]:
time_model.save(
    "interent_access_model", overwrite=True, include_optimizer=True, save_format=None,
    signatures=None, options=None
)

INFO:tensorflow:Assets written to: interent_access_model/assets


In [ ]:
model = tf.keras.models.load_model("/content/interent_access_model", custom_objects={'loss': loss})